This is a setup/test/demonstration notebook for the `AutoExposure` effect in Scopesim. The notebook uses the `irdb/METIS` configuration. The observed source is blank sky, except for the last example where a star of 0 mag is used (Vega).

In [1]:
import scopesim as sim
sim.bug_report()

# Edit this path if you have a custom install directory, otherwise comment it out.
sim.rc.__config__["!SIM.file.local_packages_path"] = "../../../../"

Python:
 3.6.8 (tags/v3.6.8:3c6b436a57, Dec 24 2018, 00:16:47) [MSC v.1916 64 bit (AMD64)]

scopesim :  0.4.0
numpy :  1.19.5
scipy :  1.5.4
astropy :  4.1
matplotlib :  3.3.4
synphot :  1.0.1
skycalc_ipy : version number not available
requests :  2.25.1
bs4 :  4.9.3
yaml :  5.4.1

Operating system:  Windows
         Release:  10
         Version:  10.0.19041
         Machine:  AMD64


If you haven't got the instrument packages yet, uncomment the following cell. 

In [2]:
# sim.download_package(["instruments/METIS", "telescopes/ELT", "locations/Armazones"])

# Imaging LM-band

In [3]:
cmd = sim.UserCommands(use_instrument="METIS", set_modes="img_lm")

In [4]:
metis = sim.OpticalTrain(cmd)

In [5]:
src = sim.source.source_templates.empty_sky()

In [6]:
metis.observe(src)

In [7]:
outimg = metis.readout()[0][1].data 
outimg /= sim.utils.from_currsys(metis['summed_exposure'].meta['ndit'])

full_well = sim.utils.from_currsys("!DET.full_well")
print("\nResult\n======")
print("Maximum value in readout (per DIT): {:8.1f}".format(outimg.max()))
print("Detector full well:                 {:8.1f}".format(full_well))
print("Fill fraction:                      {:8.1f} per cent".format(100 * outimg.max()/ full_well))

Requested exposure time: 1.000 s
Exposure parameters:
                DIT: 0.250 s  NDIT: 4
Total exposure time: 1.000 s

Result
Maximum value in readout (per DIT):  63537.6
Detector full well:                 100000.0
Fill fraction:                          63.5 per cent


Exposure time can be changed with an argument to `metis.readout()`:

In [8]:
outimg = metis.readout(exptime = 1000)[0][1].data 
outimg /= sim.utils.from_currsys(metis['summed_exposure'].meta['ndit'])

full_well = sim.utils.from_currsys("!DET.full_well")
print("\nResult\n======")
print("Maximum value in readout (per DIT): {:8.1f}".format(outimg.max()))
print("Detector full well:                 {:8.1f}".format(full_well))
print("Fill fraction:                      {:8.1f} per cent".format(100 * outimg.max()/ full_well))

Requested exposure time: 1000.000 s
Exposure parameters:
                DIT: 0.298 s  NDIT: 3354
Total exposure time: 1000.000 s

Result
Maximum value in readout (per DIT):  75006.0
Detector full well:                 100000.0
Fill fraction:                          75.0 per cent


# Imaging N-band

In [9]:
cmd = sim.UserCommands(use_instrument="METIS", set_modes=['img_n'])

In [10]:
metis = sim.OpticalTrain(cmd)

In [11]:
metis.observe(src)

In [12]:
outimg = metis.readout()[0][1].data 
outimg /= sim.utils.from_currsys(metis['summed_exposure'].meta['ndit'])

full_well = sim.utils.from_currsys("!DET.full_well")
print("\nResult\n======")
print("Maximum value in readout (per DIT): {:9.1f}".format(outimg.max()))
print("Detector full well:                 {:9.1f}".format(full_well))
print("Fill fraction:                      {:9.1f} per cent".format(100 * outimg.max()/ full_well))

Requested exposure time: 1.000 s
Exposure parameters:
                DIT: 0.017 s  NDIT: 59
Total exposure time: 1.000 s

Result
Maximum value in readout (per DIT): 2091735.2
Detector full well:                 2800000.0
Fill fraction:                           74.7 per cent


# Long-slit spectroscopy

In [13]:
cmd = sim.UserCommands(use_instrument="METIS", set_modes=['lss_l'])

In [14]:
metis = sim.OpticalTrain(cmd)

In [15]:
metis.observe(src)

In [16]:
outimg = metis.readout(exptime=3600.)[0][1].data 
outimg /= sim.utils.from_currsys(metis['summed_exposure'].meta['ndit'])
full_well = sim.utils.from_currsys("!DET.full_well")

print("\nResult\n======")
print("Maximum value in readout (per DIT): {:8.1f}".format(outimg.max()))
print("Detector full well:                 {:8.1f}".format(full_well))
print("Fill fraction:                      {:8.1f} per cent".format(100 * outimg.max()/ full_well))

Requested exposure time: 3600.000 s
Exposure parameters:
                DIT: 6.040 s  NDIT: 596
Total exposure time: 3600.000 s

Result
Maximum value in readout (per DIT):  74937.2
Detector full well:                 100000.0
Fill fraction:                          74.9 per cent


# What happens when the source saturates the detector?
Use N-band imaging of Vega. DIT is automatically set to the minimum possible value, but the centre of the star still saturates the detector. In the final image, the star's profile is capped at the full well of the detector.

In [17]:
cmd = sim.UserCommands(use_instrument="METIS", set_modes=["img_n"])

In [ ]:
metis = sim.OpticalTrain(cmd)

In [ ]:
src = sim.source.source_templates.star()

In [ ]:
metis.observe(src)

In [ ]:
outimg = metis.readout()[0][1].data
outimg /= sim.utils.from_currsys(metis['summed_exposure'].meta['ndit'])
full_well = sim.utils.from_currsys("!DET.full_well")

print("\nResult\n======")
print("Maximum value in readout (per DIT): {:9.1f}".format(outimg.max()))
print("Detector full well:                 {:9.1f}".format(full_well))
print("Fill fraction:                      {:9.1f} per cent".format(100 * outimg.max()/ full_well))

Plot a cut through the star to show how its peak saturates the detector.

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
plt.plot(outimg[950:1100, 1024])

In [ ]:
npix = (outimg >= full_well).sum()
print("Number of saturated pixels:", npix)